#☄️ Meteorite Data Exploration

##Load the Dataset

In [125]:
import pandas as pd
import numpy as np

In [126]:
meteorite_data_df = pd.read_excel('/content/Meteorite Exploration Data.xlsx')
meteorite_data_df.head(5)

,Id,Fall,Id.1,Name,Nametype,Recclass,Location,Year,Mass
0,row-hzx5-myxf_ktcf,Fell,1,Aachen,Valid,L5,50.775 6.08333,1880-01-01T00:00:00.000,21.0
1,row-rcr3.5w34_59cn,Fell,2,Aarhus,Valid,H6,56.18333 10.23333,1951-01-01T00:00:00.000,720.0
2,row-gffa_citp-4gu8,Fell,6,Abee,Valid,EH4,54.21667 -113,1952-01-01T00:00:00.000,107000.0
3,row-tw78-y449.cqkk,Fell,10,Acapulco,Valid,Acapulcoite,16.88333 -99.9,1976-01-01T00:00:00.000,1914.0
4,row-adit.5t7p.agsg,Fell,370,Achiras,Valid,L6,-33.16667 -64.95,1902-01-01T00:00:00.000,780.0


##Data Exploration

In [127]:
meteorite_data_df.shape

print(f"The dataset has {meteorite_data_df.shape[0]} rows and {meteorite_data_df.shape[1]} columns.")

The dataset has 45716 rows and 9 columns.


In [128]:
meteorite_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45716 entries, 0 to 45715
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Id        45716 non-null  object 
 1   Fall      45716 non-null  object 
 2   Id.1      45716 non-null  int64  
 3   Name      45716 non-null  object 
 4   Nametype  45716 non-null  object 
 5   Recclass  45716 non-null  object 
 6   Location  45716 non-null  object 
 7   Year      45425 non-null  object 
 8   Mass      45585 non-null  float64
dtypes: float64(1), int64(1), object(7)
memory usage: 3.1+ MB


In [129]:
meteorite_data_df.isnull().sum()

,0
Id,0
Fall,0
Id.1,0
Name,0
Nametype,0
Recclass,0
Location,0
Year,291
Mass,131


In [130]:
meteorite_data_df.iloc[1199,6]

' '

In exploring the dataset by opening it on a Spreadsheet, I could tell there are empty values in the 'Location' column.

However, this doesn't show by doing "meteorite_data_df.isnull().sum()" reason being pandas treats the following as valid strings, not missing data.

- " " (space)

- "" (empty string)

- " " (multiple spaces)

- "NaN" (string literal)

- "None"


Only actual Python-level missing values are detected ie.

- np.nan

- None

The 'Location' column contains strings that look empty but aren’t null.

Hence, the function below to identify them.

In [131]:

import pandas as pd

def count_missing_like(series):
    return series.isna().sum() + (series.astype(str).str.strip() == '').sum()

missing_df_values_summary = meteorite_data_df.apply(count_missing_like)
missing_df_values_summary

,0
Id,0
Fall,0
Id.1,0
Name,0
Nametype,0
Recclass,0
Location,7315
Year,291
Mass,131


To convert the empty String values to actual NaN values (in the entire dataframe just to be safe)

In [132]:
meteorite_data_df = meteorite_data_df.replace(r'^\s*$', np.nan, regex=True)

In [133]:
meteorite_data_df.isnull().sum()

,0
Id,0
Fall,0
Id.1,0
Name,0
Nametype,0
Recclass,0
Location,7315
Year,291
Mass,131


##Data Cleaning & Transformation

###Standardize column names

In [134]:
meteorite_data_df.columns = [c.strip().lower().replace(" ", "_") for c in meteorite_data_df.columns]
meteorite_data_df.columns


Index(['id', 'fall', 'id.1', 'name', 'nametype', 'recclass', 'location',
       'year', 'mass'],
      dtype='object')

###Split the 'recclass' into 'reclat' and 'reclong'.

The data in this field looks like '13.71111 45.17' implying these are latitudinal and longitudinal coordinates. Hence, we clean and split them to their individual columns.

In [135]:
meteorite_data_df['location'] = meteorite_data_df['location'].astype(str).str.strip()

latlong = meteorite_data_df['location'].str.split(r"\s+", n=1, expand=True)

meteorite_data_df['reclat'] = pd.to_numeric(latlong[0], errors='coerce')
meteorite_data_df['reclong'] = pd.to_numeric(latlong[1], errors='coerce')

meteorite_data_df.head(2)

,id,fall,id.1,name,nametype,recclass,location,year,mass,reclat,reclong
0,row-hzx5-myxf_ktcf,Fell,1,Aachen,Valid,L5,50.775 6.08333,1880-01-01T00:00:00.000,21.0,50.77500,6.08333
1,row-rcr3.5w34_59cn,Fell,2,Aarhus,Valid,H6,56.18333 10.23333,1951-01-01T00:00:00.000,720.0,56.18333,10.23333


###Derive the actual year values from the column 'year'

The data field currently has values like "2000-01-01T00:00:00.000", however, we would make due with the actual year value ie '2000'

In [136]:
def extract_year(x):
    if pd.isna(x):
        return np.nan
    try:
        return pd.to_datetime(x).year
    except:
        return np.nan

meteorite_data_df['year_clean'] = (
    meteorite_data_df['year']
    .apply(extract_year)
    .astype('Int64')
)

meteorite_data_df.head(2)

,id,fall,id.1,name,nametype,recclass,location,year,mass,reclat,reclong,year_clean
0,row-hzx5-myxf_ktcf,Fell,1,Aachen,Valid,L5,50.775 6.08333,1880-01-01T00:00:00.000,21.0,50.77500,6.08333,1880
1,row-rcr3.5w34_59cn,Fell,2,Aarhus,Valid,H6,56.18333 10.23333,1951-01-01T00:00:00.000,720.0,56.18333,10.23333,1951


###Clean the 'mass' column and also create the 'mass_kg' and 'log_mass' columns.

The 'log_mass' is important as it helps visualize extremely skewed mass values.

In [137]:
meteorite_data_df['mass'] = pd.to_numeric(meteorite_data_df['mass'].astype(str).str.replace(",", "").str.strip(),
                           errors='coerce')

meteorite_data_df['mass_kg'] = meteorite_data_df['mass'] / 1000

meteorite_data_df['log_mass'] = meteorite_data_df['mass'].apply(
    lambda x: np.log10(x) if pd.notna(x) and x > 0 else np.nan
)

meteorite_data_df.head(2)

,id,fall,id.1,name,nametype,recclass,location,year,mass,reclat,reclong,year_clean,mass_kg,log_mass
0,row-hzx5-myxf_ktcf,Fell,1,Aachen,Valid,L5,50.775 6.08333,1880-01-01T00:00:00.000,21.0,50.77500,6.08333,1880,0.021,1.322219
1,row-rcr3.5w34_59cn,Fell,2,Aarhus,Valid,H6,56.18333 10.23333,1951-01-01T00:00:00.000,720.0,56.18333,10.23333,1951,0.720,2.857332


###Check for unique values in the 'Fall' and 'Nametype' columns

In [138]:
print(f"The unique values in the column 'fall' are {meteorite_data_df['fall'].unique()}")

print(f"The unique values in the column 'nametype' are {meteorite_data_df['nametype'].unique()}")

print("There are no inconsistencies in the values hence we can retain them as is.")

The unique values in the column 'fall' are ['Fell' 'Found']
The unique values in the column 'nametype' are ['Valid' 'Relict']
There are no inconsistencies in the values hence we can retain them as is.


###Create is_fell (binary version)

For any match operations that might be needed in PowerBI

In [139]:
meteorite_data_df['is_fell'] = meteorite_data_df['fall'].apply(
    lambda v: 1 if v == 'Fell' else 0
)

meteorite_data_df.head(2)

,id,fall,id.1,name,nametype,recclass,location,year,mass,reclat,reclong,year_clean,mass_kg,log_mass,is_fell
0,row-hzx5-myxf_ktcf,Fell,1,Aachen,Valid,L5,50.775 6.08333,1880-01-01T00:00:00.000,21.0,50.77500,6.08333,1880,0.021,1.322219,1
1,row-rcr3.5w34_59cn,Fell,2,Aarhus,Valid,H6,56.18333 10.23333,1951-01-01T00:00:00.000,720.0,56.18333,10.23333,1951,0.720,2.857332,1


###Create a 'decade' column that'll help us better visualize patterns over time

In [140]:
meteorite_data_df['decade'] = (
    (meteorite_data_df['year_clean'] // 10) * 10
).astype('Int64')

meteorite_data_df.head(2)

,id,fall,id.1,name,nametype,recclass,location,year,mass,reclat,reclong,year_clean,mass_kg,log_mass,is_fell,decade
0,row-hzx5-myxf_ktcf,Fell,1,Aachen,Valid,L5,50.775 6.08333,1880-01-01T00:00:00.000,21.0,50.77500,6.08333,1880,0.021,1.322219,1,1880
1,row-rcr3.5w34_59cn,Fell,2,Aarhus,Valid,H6,56.18333 10.23333,1951-01-01T00:00:00.000,720.0,56.18333,10.23333,1951,0.720,2.857332,1,1950


###Drop the unwanted columns

ie. 'id', 'year'

In [141]:
meteorite_data_df.drop(columns=['id', 'year'], inplace=True)
meteorite_data_df.head(2)



,fall,id.1,name,nametype,recclass,location,mass,reclat,reclong,year_clean,mass_kg,log_mass,is_fell,decade
0,Fell,1,Aachen,Valid,L5,50.775 6.08333,21.0,50.77500,6.08333,1880,0.021,1.322219,1,1880
1,Fell,2,Aarhus,Valid,H6,56.18333 10.23333,720.0,56.18333,10.23333,1951,0.720,2.857332,1,1950


###Rename some existing columns

In [142]:
meteorite_data_df.rename(columns={
    'id.1': 'id',
    'year_clean': 'year'
}, inplace=True)

meteorite_data_df.head(2)

,fall,id,name,nametype,recclass,location,mass,reclat,reclong,year,mass_kg,log_mass,is_fell,decade
0,Fell,1,Aachen,Valid,L5,50.775 6.08333,21.0,50.77500,6.08333,1880,0.021,1.322219,1,1880
1,Fell,2,Aarhus,Valid,H6,56.18333 10.23333,720.0,56.18333,10.23333,1951,0.720,2.857332,1,1950


###Re-order the columns in the dataframe



In [143]:
meteorite_data_df = meteorite_data_df[['id', 'fall', 'is_fell', 'name', 'nametype', 'recclass', 'location', 'reclat', 'reclong', 'mass', 'mass_kg', 'log_mass', 'year', 'decade']]

meteorite_data_df.head(2)

,id,fall,is_fell,name,nametype,recclass,location,reclat,reclong,mass,mass_kg,log_mass,year,decade
0,1,Fell,1,Aachen,Valid,L5,50.775 6.08333,50.77500,6.08333,21.0,0.021,1.322219,1880,1880
1,2,Fell,1,Aarhus,Valid,H6,56.18333 10.23333,56.18333,10.23333,720.0,0.720,2.857332,1951,1950


##Export the data as a csv file for visaulization on PowerBI 🤗

In [144]:
meteorite_data_df.to_csv('Processed_Meteorite_Exploration_Data.csv')